In [ ]:
#!/usr/bin/env python
import sys
import rospy
import numpy as np
from geometry_msgs.msg import Pose, Point, Quaternion
from std_msgs.msg import String, Float32

from sdf_mp_integration.env_manager import EnvManager


ind_waypoints_dict = {1: [np.array([  [1.0, 0.0, 0.0, 0.0], 
                                    [1.0, 1.0, 0.0, 3.0], 
                                    [2.0, 1.0, 0.0, 7.0]], dtype=float)],
                     2: [np.array([  [3.5, 1.0, 0.0, 0.0], 
                                    [0.0, -1.0, 0.0, 15.0]], dtype=float)],
                     3: [np.array([  [2.5, 0.0, 0.0, 0.0], 
                                    [1.0, 0.0, 0.0, 10.0]], dtype=float)],
                    4: [np.array([  [2.5, 1.0, 0.0, 0.0], 
                                    [1.5, 0, 0.0, 10.0]], dtype=float), 
                       np.array([  [4.5, -1.0, 0.0, 0.0], 
                                    [2.0, -1.2, 0.0, 10.0]], dtype=float)]}

    
class DynamicObstacleManager:
    def __init__(self, ind_waypoints_dict):
        self.freq = 20
        self.rate = rospy.Rate(self.freq)
        self.ori = [0,0,0,0]

        # Instantiate the environment manager
        self.environment_manager = EnvManager()
        
        # Precalcuate all of our trajectories, ready for execution
        self.exps = {}
        for key, value in ind_waypoints_dict.items():
            trajectories = []
            for i in range(len(value)):
                trajectories.append(self.calculate_trajectory(value[i]))
            self.exps[key] = trajectories
#         print self.exps
        rospy.Subscriber("start_moving_obstacle", Float32, self.callback)
        rospy.loginfo("DynamicObstacleManager ready.")


    def calculate_trajectory(self, waypoints):
        trajectory = []
        pos = waypoints[0, 0:3]
        
        for i in range(len(waypoints) - 1):
            delta_t = waypoints[i + 1, 3] - waypoints[i, 3]
            num_steps  = int(delta_t * self.freq)
            delta_x = (waypoints[i + 1, 0:3] - waypoints[i, 0:3])/num_steps
            for j in range(num_steps):
                pos = pos + delta_x
                trajectory.append(pos)
                
#         print np.array(trajectory)
        return np.array(trajectory)

    def move_to_init(self, exp_id):
        trajectories = self.exps[exp_id]
        self.environment_manager.move_object('cylinder', trajectories[0][0, :], self.ori)
        if(len(trajectories) > 1):
            self.environment_manager.move_object('cylinder2', trajectories[1][0, :], self.ori)
        
    def execute_trajectories(self, trajectories):
        num_traj_points = trajectories[0].shape[0]
        for i in range(num_traj_points):
            self.environment_manager.move_object('cylinder', trajectories[0][i, :], self.ori)
            if(len(trajectories) > 1):
                self.environment_manager.move_object('cylinder2', trajectories[1][i, :], self.ori)
            self.rate.sleep()

    def execute_trajectory_id(self, exp_id):
        self.execute_trajectories(self.exps[exp_id])
        
    def callback(self, data):    
        # pick and place params
        start_pos = np.array([0.4,2.0,0])
        ori = [0,0,0,0]
        velocity = [0, -data.data, 0]

        self.rate.sleep()

        for i in range(duration*self.freq):
            print('moving')
            fin_pos = start_pos + (i+1)*np.array(velocity)/self.freq
            self.environment_manager.move_object('cylinder', fin_pos, ori)
            self.rate.sleep()
        
        rospy.sleep(1)
        self.environment_manager.move_object('cylinder', start_pos, ori)


In [ ]:
rospy.init_node('moving_obstacle_node', anonymous=True)
obstacle_manager = DynamicObstacleManager(ind_waypoints_dict)


In [ ]:
exp_id = 4

In [ ]:
obstacle_manager.move_to_init(exp_id)

In [ ]:
obstacle_manager.execute_trajectory_id(exp_id)